### DATA STATION PREPROCESSING

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import geopandas as gpd
from shapely.geometry import box
import os

In [2]:
df = pd.read_csv('data_stasiun.csv')

df = df[(df['year'] >= 1995) & (df['year'] <= 2014)]
df = df.dropna(subset=['lat', 'long', 'elev'])
df = df.rename(columns={'long': 'lon'})

#### FILTER DATA TAHUN 1995-2014

In [3]:
df.to_csv('data_stasiun_1995_2014.csv', index=False)

In [5]:
monthly_df = (
    df.groupby(['stanam', 'lon', 'lat', 'elev', 'year', 'month'], as_index=False)['rainfall'].sum(min_count=1).rename(
        columns={'rainfall': 'monthly_rainfall'})
)
#monthly_df.to_csv('data_stasiun_bulanan_1995_2014.csv', index=False)

#### FILTER DATA STASIUN > 70% dan DATA STASIUN <= 70%

In [ ]:
#monthly_df["date"] = pd.to_datetime(df[["year", "month"]])

# --- Filter stations with >50% filled Rainfall data ---
# Replace 'station_id' with your actual station identifier column
station_col = 'stanam'  # Change this if your column has a different name

# Compute the fraction of non-NaN Rainfall data for each station
station_counts = monthly_df.groupby(station_col)['monthly_rainfall'].apply(lambda x: x.notna().mean())
good_mon_stations = station_counts[station_counts > 0.7].index

# Filter the DataFrame to keep only good stations
filter_mon_df = monthly_df[monthly_df[station_col].isin(good_mon_stations)].copy()

# (Optional) Save the filtered data
#filtered_df.to_csv('good_mon_station.csv', index=False)

In [ ]:
bad_mon_stations = station_counts[station_counts <= 0.7].index
excluded_mon_df = monthly_df[monthly_df[station_col].isin(bad_mon_stations)].copy()

# (Optional) Save the excluded data
#excluded_df.to_csv('bad_mon_station.csv', index=False)

#### NANs DATA FILLING

In [13]:
mswep_interp_df = pd.read_csv("mswep_at_stations_timeseries.csv")

good_sta_df = filter_mon_df.merge(
    mswep_interp_df, on=['stanam', 'year', 'month'], how='left'
)
bad_sta_df = excluded_mon_df.merge(
    mswep_interp_df, on=['stanam', 'year', 'month'], how='left'
)

In [14]:
good_sta_df = good_sta_df.rename(columns={
    'precip_interp': 'mswep_precip',
    'lon_x': 'lon',
    'lat_x': 'lat',
    'elev_x': 'elev'
})
good_sta_df = good_sta_df[['stanam', 'lon', 'lat', 'elev', 'year', 'month', 'monthly_rainfall', 'mswep_precip']]
#good_sta_df.to_csv("good_station_mswp.csv", index=False)
#good_sta_df

In [15]:
bad_sta_df = bad_sta_df.rename(columns={
    'precip_interp': 'mswep_precip',
    'lon_x': 'lon',
    'lat_x': 'lat',
    'elev_x': 'elev'
})
bad_sta_df = bad_sta_df[['stanam', 'lon', 'lat', 'elev', 'year', 'month', 'monthly_rainfall', 'mswep_precip']]
#bad_sta_df.to_csv("bad_station_mswp.csv", index=False)
#bad_sta_df

##### GOOD STATION FILLING

In [16]:
# Calculate monthly climatology for each station
climatology = good_sta_df.groupby(['stanam', 'month'], as_index=False)['monthly_rainfall'].mean()
climatology = climatology.rename(columns={'monthly_rainfall': 'clim_mean'})

# Ensure 'stanam' and 'month' are the same type in both dataframes
good_sta_df['stanam'] = good_sta_df['stanam'].astype(climatology['stanam'].dtype)
good_sta_df['month'] = good_sta_df['month'].astype(climatology['month'].dtype)

# Merge climatology back to the main dataframe
good_sta_df = good_sta_df.merge(climatology, on=['stanam', 'month'], how='left')

# Fill missing values with climatology
if 'clim_mean' in good_sta_df.columns:
	good_sta_df['rain_hist_filled'] = good_sta_df['monthly_rainfall'].combine_first(good_sta_df['clim_mean'])
else:
	print("Warning: 'clim_mean' column not found after merge.")
	good_sta_df['rain_hist_filled'] = good_sta_df['monthly_rainfall']

In [17]:
def spatial_fill(df, value_col='rain_hist_filled', group_col='month', lat_col='lat', lon_col='lon', n_neighbors=10):
    filled = df.copy()
    for idx, row in filled[filled[value_col].isna()].iterrows():
        # Find same month, other stations, not NaN
        candidates = df[(df[group_col] == row[group_col]) & (df['stanam'] != row['stanam']) & df[value_col].notna()]
        if not candidates.empty:
            # Compute distance to all candidates
            dists = np.sqrt((candidates[lat_col] - row[lat_col])**2 + (candidates[lon_col] - row[lon_col])**2)
            n = min(n_neighbors, len(dists))
            if n > 0:
                nearest = candidates.loc[dists.nsmallest(n).index]
                filled.at[idx, value_col] = nearest[value_col].mean()
    return filled

good_sta_df = spatial_fill(good_sta_df, value_col='rain_hist_filled')

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Only compare where both are not NaN
good_sta_df = good_sta_df.rename(columns={'rain_hist_filled': 'filled_rainfall'})
mask = good_sta_df['monthly_rainfall'].notna() & good_sta_df['mswep_precip'].notna()
obs = good_sta_df.loc[mask, 'filled_rainfall']
mswep = good_sta_df.loc[mask, 'mswep_precip']

r2 = r2_score(obs, mswep)
rmse = np.sqrt(mean_squared_error(obs, mswep))

print(f"R-squared: {r2:.3f}")
print(f"RMSE: {rmse:.2f}")

In [ ]:
import numpy as np
from scipy import stats

def quantile_mapping(obs, ref, target):
    obs = np.array(obs)
    ref = np.array(ref)
    target = np.array(target)
    valid = ~np.isnan(obs) & ~np.isnan(ref)
    obs = obs[valid]
    ref = ref[valid]
    # Sort and get quantiles
    quantiles = np.linspace(0, 1, len(obs))
    obs_sorted = np.sort(obs)
    ref_sorted = np.sort(ref)
    # For each target value, get its quantile in obs, then map to ref
    target_q = stats.rankdata(target, method='average') / (len(target) + 1)
    mapped = np.interp(target_q, quantiles, ref_sorted, left=ref_sorted[0], right=ref_sorted[-1])
    mapped[np.isnan(target)] = np.nan
    return mapped

# Example for one month (repeat for each month)
for stanam in good_sta_df['stanam'].unique():
    for m in range(1, 13):
        mask = (good_sta_df['stanam'] == stanam) & (good_sta_df['month'] == m)
        obs = good_sta_df.loc[mask, 'filled_rainfall']
        ref = good_sta_df.loc[mask, 'mswep_precip']
        target = good_sta_df.loc[mask, 'filled_rainfall']
        if obs.notna().sum() > 10 and ref.notna().sum() > 10:  # Only if enough data
            mapped = quantile_mapping(obs, ref, target)
            good_sta_df.loc[mask, 'filled_rainfall_bc_qm'] = mapped

In [ ]:
# 3. Check R-squared and RMSE after bias correction (where both observed and bias-corrected filled_rainfall are available)
mask_bc = good_sta_df['mswep_precip'].notna() & good_sta_df['filled_rainfall_bc_qm'].notna()
obs_bc = good_sta_df.loc[mask_bc, 'mswep_precip']
filled_bc = good_sta_df.loc[mask_bc, 'filled_rainfall_bc_qm']

from sklearn.metrics import r2_score, mean_squared_error
r2_bc = r2_score(filled_bc, obs_bc)
rmse_bc = np.sqrt(mean_squared_error(filled_bc, obs_bc))

print(f"Bias-corrected filled_rainfall R-squared: {r2_bc:.3f}")
print(f"Bias-corrected filled_rainfall RMSE: {rmse_bc:.2f}")

##### BAD STATION FILLING

In [22]:
# Calculate monthly climatology for each station
climatology = bad_sta_df.groupby(['stanam', 'month'], as_index=False)['monthly_rainfall'].mean()
climatology = climatology.rename(columns={'monthly_rainfall': 'clim_mean'})

# Ensure 'stanam' and 'month' are the same type in both dataframes
bad_sta_df['stanam'] = bad_sta_df['stanam'].astype(climatology['stanam'].dtype)
bad_sta_df['month'] = bad_sta_df['month'].astype(climatology['month'].dtype)

# Merge climatology back to the main dataframe
bad_sta_df = bad_sta_df.merge(climatology, on=['stanam', 'month'], how='left')

# Fill missing values with climatology
if 'clim_mean' in bad_sta_df.columns:
	bad_sta_df['rain_hist_filled'] = bad_sta_df['monthly_rainfall'].combine_first(bad_sta_df['clim_mean'])
else:
	print("Warning: 'clim_mean' column not found after merge.")
	bad_sta_df['rain_hist_filled'] = bad_sta_df['monthly_rainfall']

In [ ]:
bad_sta_df = spatial_fill(bad_sta_df, value_col='rain_hist_filled')

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Only compare where both are not NaN
bad_sta_df = bad_sta_df.rename(columns={'rain_hist_filled': 'filled_rainfall'})
mask = bad_sta_df['monthly_rainfall'].notna() & bad_sta_df['mswep_precip'].notna()
obs = bad_sta_df.loc[mask, 'filled_rainfall']
mswep = bad_sta_df.loc[mask, 'mswep_precip']

r2 = r2_score(obs, mswep)
rmse = np.sqrt(mean_squared_error(obs, mswep))

print(f"R-squared: {r2:.3f}")
print(f"RMSE: {rmse:.2f}")

In [ ]:
import numpy as np
from scipy import stats

# Example for one month (repeat for each month)
for stanam in bad_sta_df['stanam'].unique():
    for m in range(1, 13):
        mask = (bad_sta_df['stanam'] == stanam) & (bad_sta_df['month'] == m)
        obs = bad_sta_df.loc[mask, 'filled_rainfall']
        ref = bad_sta_df.loc[mask, 'mswep_precip']
        target = bad_sta_df.loc[mask, 'filled_rainfall']
        if obs.notna().sum() > 10 and ref.notna().sum() > 10:  # Only if enough data
            mapped = quantile_mapping(obs, ref, target)
            bad_sta_df.loc[mask, 'filled_rainfall_bc_qm'] = mapped

In [ ]:
# 3. Check R-squared and RMSE after bias correction (where both observed and bias-corrected filled_rainfall are available)
mask_bc = bad_sta_df['mswep_precip'].notna() & bad_sta_df['filled_rainfall_bc_qm'].notna()
obs_bc = bad_sta_df.loc[mask_bc, 'mswep_precip']
filled_bc = bad_sta_df.loc[mask_bc, 'filled_rainfall_bc_qm']

from sklearn.metrics import r2_score, mean_squared_error
r2_bc = r2_score( filled_bc, obs_bc)
rmse_bc = np.sqrt(mean_squared_error(filled_bc, obs_bc))

print(f"Bias-corrected filled_rainfall R-squared: {r2_bc:.3f}")
print(f"Bias-corrected filled_rainfall RMSE: {rmse_bc:.2f}")

##### MERGE DATAFRAME AND QUALITY CONTROL

In [ ]:
merged_df = pd.concat([good_sta_df, bad_sta_df], ignore_index=True)
#merged_df.to_csv("merged_station_mswp.csv", index=False)
#merged_df = merged_df.drop(columns=['clim_mean'])
merged_df

In [45]:
# Example: Remove negative or extremely high values
qc_df = merged_df.copy()
qc_df.loc[(qc_df['filled_rainfall_bc_qm'] < 0) | (qc_df['filled_rainfall_bc_qm'] > 1000), 'filled_rainfall_bc_qm'] = np.nan

In [46]:
# Example: Flag zero rainfall in wet months (customize months for your region)
wet_months = [11, 12, 1, 2, 3, 4]  # Example for Indonesia
qc_df['qc_flag'] = 0
qc_df.loc[(qc_df['month'].isin(wet_months)) & (qc_df['filled_rainfall_bc_qm'] == 0), 'qc_flag'] = 1  # Flag as suspicious

In [47]:
# Calculate climatology stats
clim_stats = qc_df.groupby(['stanam', 'month'])['filled_rainfall_bc_qm'].agg(['mean', 'std']).reset_index()
qc_df = qc_df.merge(clim_stats, on=['stanam', 'month'], how='left')
# Flag outliers
qc_df['outlier_flag'] = ((qc_df['filled_rainfall_bc_qm'] - qc_df['mean']).abs() > 3 * qc_df['std']).astype(int)

In [ ]:
duplicates = qc_df.duplicated(subset=['stanam', 'year', 'month'])
qc_df = qc_df[~duplicates]
missing_counts = qc_df.groupby('stanam')['filled_rainfall_bc_qm'].apply(lambda x: x.isna().sum())
print(missing_counts)

In [ ]:
# 1. Fill with bias-corrected MSWEP where available
qc_df['final_filled'] = qc_df['filled_rainfall_bc_qm'].combine_first(qc_df['mswep_precip'])

# 2. (Optional) If still missing, fill with station climatology
if 'mean' in qc_df.columns:
    qc_df['final_filled'] = qc_df['final_filled'].combine_first(qc_df['mean'])

# 3. (Optional) If still missing, fill with other logic or leave as NaN

# Check remaining missing values
print(qc_df['final_filled'].isna().sum())

In [ ]:
import matplotlib.pyplot as plt

# Calculate metrics
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# For a single station
station_name = qc_df['stanam'].unique()[0]
df_plot = qc_df[qc_df['stanam'] == station_name].copy()
df_plot['date'] = pd.to_datetime(df_plot[['year', 'month']].assign(day=1))

# Metrics for subplot 1 (Filled Rainfall vs MSWEP)
mask_filled = df_plot['filled_rainfall'].notna() & df_plot['mswep_precip'].notna()
r2_filled = r2_score(df_plot.loc[mask_filled, 'filled_rainfall'], df_plot.loc[mask_filled, 'mswep_precip'])
rmse_filled = np.sqrt(mean_squared_error(df_plot.loc[mask_filled, 'filled_rainfall'], df_plot.loc[mask_filled, 'mswep_precip']))

# Metrics for subplot 2 (Final Filled vs MSWEP)
mask_final = df_plot['final_filled'].notna() & df_plot['mswep_precip'].notna()
r2_final = r2_score(df_plot.loc[mask_final, 'final_filled'], df_plot.loc[mask_final, 'mswep_precip'])
rmse_final = np.sqrt(mean_squared_error(df_plot.loc[mask_final, 'final_filled'], df_plot.loc[mask_final, 'mswep_precip']))

# Metrics for subplot 3 (Filled Rainfall vs Final Rainfall)
mask_ff = df_plot['filled_rainfall'].notna() & df_plot['final_filled'].notna()
r2_ff = r2_score(df_plot.loc[mask_ff, 'filled_rainfall'], df_plot.loc[mask_ff, 'final_filled'])
rmse_ff = np.sqrt(mean_squared_error(df_plot.loc[mask_ff, 'filled_rainfall'], df_plot.loc[mask_ff, 'final_filled']))

fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

axs[0].plot(df_plot['date'], df_plot['mswep_precip'], color='gray', linestyle='--', label='MSWEP Precip')
axs[0].plot(df_plot['date'], df_plot['filled_rainfall'], color='tab:blue', label='Filled Rainfall')
axs[0].set_ylabel('Precipitation (mm/month)')
axs[0].set_title(f'Station Filled vs MSWEP Precipitation for {station_name}')
axs[0].grid(True)
axs[0].legend()
axs[0].text(0.01, 0.95, f'RMSE={rmse_filled:.2f}\nR²={r2_filled:.3f}', transform=axs[0].transAxes,
            fontsize=10, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

axs[1].plot(df_plot['date'], df_plot['mswep_precip'], color='gray', linestyle='--', label='MSWEP Precip')
axs[1].plot(df_plot['date'], df_plot['final_filled'], color='tab:red', label='Final Filled Rainfall')
axs[1].set_ylabel('Precipitation (mm/month)')
axs[1].set_title(f'Final Station Filled vs MSWEP Precipitation for {station_name}')
axs[1].grid(True)
axs[1].legend()
axs[1].text(0.01, 0.95, f'RMSE={rmse_final:.2f}\nR²={r2_final:.3f}', transform=axs[1].transAxes,
            fontsize=10, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

axs[2].plot(df_plot['date'], df_plot['filled_rainfall'], color='tab:blue', label='Filled Rainfall')
axs[2].plot(df_plot['date'], df_plot['final_filled'], color='tab:red', linestyle='--', label='Final Filled Rainfall')
axs[2].set_ylabel('Precipitation (mm/month)')
axs[2].set_title(f'Station Filled vs Final Filled Rainfall for {station_name}')
axs[2].set_xlabel('Date')
axs[2].grid(True)
axs[2].legend()
axs[2].text(0.01, 0.95, f'RMSE={rmse_ff:.2f}\nR²={r2_ff:.3f}', transform=axs[2].transAxes,
            fontsize=10, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import pandas as pd

# Mean RMSE and R2 for filled_rainfall vs mswep_precip
results_filled = []
for stanam, group in qc_df.groupby('stanam'):
    mask = group['filled_rainfall'].notna() & group['mswep_precip'].notna()
    if mask.sum() > 0:
        obs = group.loc[mask, 'filled_rainfall']
        mswep = group.loc[mask, 'mswep_precip']
        rmse = np.sqrt(mean_squared_error(obs, mswep))
        r2 = r2_score(obs, mswep)
        results_filled.append({'stanam': stanam, 'rmse': rmse, 'r2': r2})
df_filled = pd.DataFrame(results_filled)
mean_rmse_filled = df_filled['rmse'].mean()
mean_r2_filled = df_filled['r2'].mean()

# Mean RMSE and R2 for final_filled vs mswep_precip
results_final = []
for stanam, group in qc_df.groupby('stanam'):
    mask = group['final_filled'].notna() & group['mswep_precip'].notna()
    if mask.sum() > 0:
        obs = group.loc[mask, 'final_filled']
        mswep = group.loc[mask, 'mswep_precip']
        rmse = np.sqrt(mean_squared_error(obs, mswep))
        r2 = r2_score(obs, mswep)
        results_final.append({'stanam': stanam, 'rmse': rmse, 'r2': r2})
df_final = pd.DataFrame(results_final)
mean_rmse_final = df_final['rmse'].mean()
mean_r2_final = df_final['r2'].mean()

print("Filled Rainfall Before Bias Correction:")
print(f"All Station - RMSE (Filled Rainfall): {mean_rmse_filled:.2f}, R2 (Filled Rainfall): {mean_r2_filled:.3f}")
print("\nFinal Rainfall After Bias Correction:")
print(f"All Station - RMSE (Final Rainfall): {mean_rmse_final:.2f}, R2 (Final Rainfall: {mean_r2_final:.3f}")

In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import griddata
from sklearn.metrics import mean_squared_error, mean_absolute_error

all_results = []

for year in range(1995, 2015):  # Loop over years
    df_year = qc_df[qc_df['year'] == year].copy()
    for month in range(1, 13):  # Loop over months
        df_month = df_year[df_year['month'] == month].copy()
        coords = df_month[['lon', 'lat']].values
        stations = df_month['stanam'].values
        mswep = df_month['mswep_precip'].values
        final_filled = df_month['final_filled'].values
        filled_rainfall = df_month['filled_rainfall'].values

        for i in range(len(df_month)):
            mask = np.arange(len(df_month)) != i
            train_coords = coords[mask]
            train_final = final_filled[mask]
            train_filled = filled_rainfall[mask]
            test_coord = coords[i].reshape(1, -1)
            true_mswep = mswep[i]

            pred_final = griddata(train_coords, train_final, test_coord, method='nearest')[0]
            pred_filled = griddata(train_coords, train_filled, test_coord, method='nearest')[0]

            # Use sklearn metrics for single value (as 1-element arrays)
            if not np.isnan(true_mswep) and not np.isnan(pred_final):
                rmse_final = mean_squared_error([true_mswep], [pred_final], squared=False)
                mae_final = mean_absolute_error([true_mswep], [pred_final])
            else:
                rmse_final = mae_final = np.nan

            if not np.isnan(true_mswep) and not np.isnan(pred_filled):
                rmse_filled = mean_squared_error([true_mswep], [pred_filled], squared=False)
                mae_filled = mean_absolute_error([true_mswep], [pred_filled])
            else:
                rmse_filled = mae_filled = np.nan

            all_results.append({
                'year': year,
                'stanam': stations[i],
                'month': month,
                'rmse_final_filled': rmse_final,
                'mae_final_filled': mae_final,
                'rmse_filled_rainfall': rmse_filled,
                'mae_filled_rainfall': mae_filled
            })

In [ ]:
skill_df = pd.DataFrame(all_results)
print(skill_df.shape)
skill_df.to_csv("skill_metrics.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Load your skill metrics data
cv_results = pd.read_csv('skill_metrics.csv')
#years = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004]
#years = [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]
#cv_results = cv_results[cv_results['year'].isin(years)]

# Create a datetime column
cv_results['date'] = pd.to_datetime(cv_results['year'].astype(str) + '-' + cv_results['month'].astype(str) + '-01')

# Group by date and calculate mean RMSE for all stations
mean_rmse = cv_results.groupby('date')[['rmse_final_filled', 'rmse_filled_rainfall']].mean().reset_index()
mean_rmse_final = mean_rmse['rmse_final_filled'].mean()
mean_rmse_filled = mean_rmse['rmse_filled_rainfall'].mean()

plt.figure(figsize=(14, 6))
plt.plot(mean_rmse['date'], mean_rmse['rmse_final_filled'], label='RMSE Final Filled')
plt.plot(mean_rmse['date'], mean_rmse['rmse_filled_rainfall'], label='RMSE Filled Rainfall')
plt.axhline(mean_rmse_final, color='k', linestyle='--', label=f'Mean RMSE Final Filled: {mean_rmse_final:.2f}')
plt.axhline(mean_rmse_filled, color='brown', linestyle='--', label=f'Mean RMSE Filled Rainfall: {mean_rmse_filled:.2f}')
plt.xlabel('Date')
plt.ylabel('RMSE')
plt.title('Time Series RMSE')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree

def compute_spatial_correlation(df, value_col, n_neighbors=3):
    results = []
    for (year, month), group in df.groupby(['year', 'month']):
        group = group.dropna(subset=[value_col, 'lon', 'lat'])
        if len(group) <= n_neighbors:
            continue
        coords = group[['lon', 'lat']].values
        values = group[value_col].values
        tree = cKDTree(coords)
        neighbor_means = []
        for i, (lon, lat) in enumerate(coords):
            dists, idxs = tree.query([lon, lat], k=n_neighbors+1)
            idxs = idxs[1:]  # exclude self
            neighbor_vals = values[idxs]
            neighbor_means.append(np.nanmean(neighbor_vals))
        mask = ~np.isnan(values) & ~np.isnan(neighbor_means)
        if mask.sum() > n_neighbors:
            corr = np.corrcoef(values[mask], np.array(neighbor_means)[mask])[0, 1]
            results.append({'year': year, 'month': month, 'spatial_corr': corr})
    return pd.DataFrame(results)

# Calculate for both columns
spatial_corr_final = compute_spatial_correlation(qc_df, value_col='final_filled', n_neighbors=3)
spatial_corr_filled = compute_spatial_correlation(qc_df, value_col='filled_rainfall', n_neighbors=3)

print("Spatial Correlation for final_filled:")
print(spatial_corr_final.head())
print("\nSpatial Correlation for filled_rainfall:")
print(spatial_corr_filled.head())

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Ensure 'year' and 'month' are integers
spatial_corr_final['year'] = spatial_corr_final['year'].astype(int)
spatial_corr_final['month'] = spatial_corr_final['month'].astype(int)
spatial_corr_filled['year'] = spatial_corr_filled['year'].astype(int)
spatial_corr_filled['month'] = spatial_corr_filled['month'].astype(int)

# Create a proper datetime column for the x-axis
spatial_corr_final['date'] = pd.to_datetime(
    spatial_corr_final['year'].astype(str) + '-' + spatial_corr_final['month'].astype(str).str.zfill(2) + '-01'
)
spatial_corr_filled['date'] = pd.to_datetime(
    spatial_corr_filled['year'].astype(str) + '-' + spatial_corr_filled['month'].astype(str).str.zfill(2) + '-01'
)
mean_corr_final = spatial_corr_final['spatial_corr'].mean()
mean_corr_filled = spatial_corr_filled['spatial_corr'].mean()

plt.figure(figsize=(14, 6))
plt.plot(spatial_corr_final['date'], spatial_corr_final['spatial_corr'], label='Final Filled Rainfall')
plt.plot(spatial_corr_filled['date'], spatial_corr_filled['spatial_corr'], label='Filled Rainfall')
plt.axhline(mean_corr_final, color='k', linestyle='--', label=f'Mean Final Filled Spatial Corr: {mean_corr_final:.2f}')
plt.axhline(mean_corr_filled, color='brown', linestyle='--', label=f'Mean Filled Spatial Corr: {mean_corr_filled:.2f}')
plt.title('Spatial Correlation of Final Filled Rainfall by Month')
plt.xlabel('Date')
plt.ylabel('Spatial Correlation')
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()

In [45]:
final_df = qc_df.drop(columns=['filled_rainfall', 'filled_rainfall_bc_qm'])
final_df = final_df.rename(columns={'final_filled': 'final_rainfall'})
final_df.to_csv("final_station_mswp.csv", index=False)

#### DATA MASKING COASTLINE

In [6]:
from scipy.interpolate import griddata

#data_folder = "DATA V3/Masked/data_v3_masked_bnds0/"
#source = "UKESM1-0-LL"  # Change to your source model
#filename = f"data_V3_{source}_Jawa_bnds0_masked.csv"
#output_folder = "DATA V3_Regrid/"
#df = pd.read_csv(data_folder + filename)
#df = df_bnds0
df = pd.read_csv("data_mswep_regrid_masked.csv")

# Define target grid (consider clipping to your data extent)
lon_new = np.arange(105, 115.01, 0.1)
lat_new = np.arange(-9, -5.79, 0.1)
lon2d, lat2d = np.meshgrid(lon_new, lat_new)

out_list = []

for t, df_t in df.groupby('time'):
    points = np.column_stack((df_t['lon'], df_t['lat']))
    values = df_t['precipitation'].values
    
    # Step 1: Linear interpolation
    grid_linear = griddata(points, values, (lon2d, lat2d), method='linear')
    
    # Step 2: Fill NaNs with nearest
    if np.isnan(grid_linear).any():
        grid_nearest = griddata(points, values, (lon2d, lat2d), method='nearest')
        grid_z = np.where(np.isnan(grid_linear), grid_nearest, grid_linear)
    else:
        grid_z = grid_linear
    
    # Store results
    out_list.append(pd.DataFrame({
        'time': t,
        'lon': lon2d.ravel(),
        'lat': lat2d.ravel(),
        'precipitation': grid_z.ravel()
    }))

df_out = pd.concat(out_list, ignore_index=True)
df_out.to_csv('data_mswep_regrid_masked.csv', index=False)

In [9]:
shp_path = "peta_shp_indonesia.shp"

# Load Indonesia shapefile and select Java Island (by bounding box or attribute)
indo = gpd.read_file(shp_path)
# If the shapefile has an attribute for island name, filter by that (example: 'Pulau' == 'Jawa')
# java = indo[indo['Pulau'] == 'Jawa']
# If not, use bounding box for Java
minx, miny, maxx, maxy = 105, -9, 115, -5.8
java = indo.cx[minx:maxx, miny:maxy]

v3_gdf = gpd.GeoDataFrame(
    df_out,
    geometry=gpd.points_from_xy(df_out['lon'], df_out['lat']),
    crs=java.crs
)

# Use spatial join for fast masking
v3_masked = gpd.sjoin(v3_gdf, java, predicate='within', how='inner')

# Drop geometry and join columns
v3_masked = v3_masked.drop(columns=['geometry', 'index_right'], errors='ignore')

In [11]:
v3_masked.to_csv("data_mswep_regrid_masked.csv", index=False)